In [2]:
import os
import json
import random

In [2]:
with open(r"G:\python_codes\Principles_of_Computer_Construction_KG\output\[banned]\re_pre_annotated_trainset_top_300.json", 'r', encoding='utf-8') as f:
    re_trainset = json.loads(f.read())

with open(r"G:\python_codes\Principles_of_Computer_Construction_KG\output\[banned]\re_pre_annotated_validset_top_50.json", 'r', encoding='utf-8') as f:
    re_validset = json.loads(f.read())

with open(r"G:\python_codes\Principles_of_Computer_Construction_KG\output\[banned]\re_pre_annotated_testset_top_100.json", 'r', encoding='utf-8') as f:
    re_testset = json.loads(f.read())

In [23]:
re_trainset[0]

{'id': 7740,
 'annotations': [{'id': 9173,
   'completed_by': 1,
   'result': [{'value': {'start': 14,
      'end': 18,
      'text': '预取结构',
      'labels': ['TECH']},
     'id': '1',
     'from_name': 'label',
     'to_name': 'text',
     'type': 'labels',
     'origin': 'prediction'},
    {'value': {'start': 0, 'end': 7, 'text': '双倍数据率结构', 'labels': ['TECH']},
     'id': '8ExFX-LBm3',
     'from_name': 'label',
     'to_name': 'text',
     'type': 'labels',
     'origin': 'manual'},
    {'from_id': '8ExFX-LBm3',
     'to_id': '1',
     'type': 'relation',
     'direction': 'left',
     'labels': ['org:contain']}],
   'was_cancelled': False,
   'ground_truth': False,
   'created_at': '2024-09-04T01:57:37.442749Z',
   'updated_at': '2024-09-04T01:57:37.442749Z',
   'draft_created_at': '2024-09-04T01:55:08.936141Z',
   'lead_time': 116.893,
   'prediction': {'id': 158575,
    'result': [{'id': '1',
      'from_name': 'label',
      'to_name': 'text',
      'type': 'labels',
      'valu

In [22]:
re_validset[0]

{'id': 11887,
 'annotations': [{'id': 9475,
   'completed_by': 1,
   'result': [{'value': {'start': 2,
      'end': 8,
      'text': 'cache行',
      'labels': ['STOR']},
     'id': '1',
     'from_name': 'label',
     'to_name': 'text',
     'type': 'labels',
     'origin': 'prediction'},
    {'value': {'start': 47, 'end': 50, 'text': '控制器', 'labels': ['COMP']},
     'id': '2',
     'from_name': 'label',
     'to_name': 'text',
     'type': 'labels',
     'origin': 'prediction'},
    {'value': {'start': 13, 'end': 17, 'text': '处理机核', 'labels': ['COMP']},
     'id': '3',
     'from_name': 'label',
     'to_name': 'text',
     'type': 'labels',
     'origin': 'prediction'},
    {'value': {'start': 24, 'end': 28, 'text': '处理机核', 'labels': ['COMP']},
     'id': '4',
     'from_name': 'label',
     'to_name': 'text',
     'type': 'labels',
     'origin': 'prediction'},
    {'from_id': '4',
     'to_id': '2',
     'type': 'relation',
     'direction': 'right',
     'labels': ['org:attribute'

In [3]:
re_testset[0]

{'id': 12406,
 'annotations': [{'id': 9525,
   'completed_by': 1,
   'result': [],
   'was_cancelled': False,
   'ground_truth': False,
   'created_at': '2024-09-15T06:23:37.669248Z',
   'updated_at': '2024-09-15T06:23:37.669248Z',
   'draft_created_at': None,
   'lead_time': 0.995,
   'prediction': {'id': 163241,
    'result': [],
    'model_version': '',
    'created_ago': '3\xa0hours, 39\xa0minutes',
    'score': 0.0,
    'cluster': None,
    'neighbors': None,
    'mislabeling': 0.0,
    'created_at': '2024-09-15T02:44:30.767675Z',
    'updated_at': '2024-09-15T02:44:30.767675Z',
    'model': None,
    'model_run': None,
    'task': 12406,
    'project': 47},
   'result_count': 0,
   'unique_id': '8fcb831e-02d3-47b5-ad35-451eb2338592',
   'import_id': None,
   'last_action': None,
   'task': 12406,
   'project': 47,
   'updated_by': 1,
   'parent_prediction': 163241,
   'parent_annotation': None,
   'last_created_by': None}],
 'file_upload': '7b052611-re_pre_annotated_testset.json'

In [4]:
# 把标注好的 RE 数据集转换为可以处理的标准数据集
def transform_dataset(dataset):
    processed_dataset = []

    for i, data in enumerate(dataset):
        new_item = {
            "sentence": dataset[i]["data"]["text"],
            "annotations": [],
            "prediction": [],
        }

        if re_trainset[i].get("annotations"):
            annotations = dataset[i]["annotations"][0]
            if annotations.get("result"):
                new_item["annotations"] += annotations["result"]
            if annotations.get("prediction") and annotations["prediction"].get(
                "result"
            ):
                new_item["prediction"] += annotations["prediction"]["result"]

        processed_dataset.append(new_item)

    assert len(dataset) == len(
        processed_dataset
    ), f"The length of input is {len(dataset)}, length of return is {len(processed_dataset)}, not equal."

    extracted_label_dataset = []

    for item in processed_dataset:
        new_item = {"sentence": item["sentence"], "entities": [], "relations": []}
        map_entities = {}
        entities = []
        relations = []

        # Extract entity labels and store them in a dictionary with their IDs
        for annotation in item["annotations"]:
            if annotation["type"] == "labels":
                map_entities[annotation["id"]] = annotation["value"]["text"]
                entities.append(
                    {
                        "text": annotation["value"]["text"],
                        "start": annotation["value"]["start"],
                        "end": annotation["value"]["end"],
                        "labels": annotation["value"]["labels"],
                    }
                )

        # Extract relationship labels and replace IDs with entity names
        for annotation in item["annotations"]:
            if annotation["type"] == "relation":
                if annotation["type"] == "relation":
                    from_entity = map_entities[annotation["from_id"]]
                    to_entity = map_entities[annotation["to_id"]]
                    relation_label = (
                        annotation["labels"][0].split(":")[1]
                        if annotation.get("labels")
                        else "related"
                    )
                    direction = annotation["direction"]

                if direction == "left":
                    triple_tuple = (from_entity, relation_label, to_entity)
                elif direction == "right":
                    triple_tuple = (to_entity, relation_label, from_entity)
                elif direction == "bi":
                    triple_tuple = (from_entity, relation_label, to_entity)
                    relations.append((to_entity, relation_label, from_entity))

                relations.append(triple_tuple)

        new_item["entities"] += entities
        new_item["relations"] += relations

        extracted_label_dataset.append(new_item)

    return extracted_label_dataset

In [5]:
trans_trainset = transform_dataset(re_trainset)
trans_validset = transform_dataset(re_validset)
trans_testset = transform_dataset(re_testset)

In [6]:
# with open(r"G:\python_codes\Principles_of_Computer_Construction_KG\output\[banned]\transformed_trainset.json", 'w', encoding='utf-8') as f:
#     f.write(json.dumps(trans_trainset))
# with open(r"G:\python_codes\Principles_of_Computer_Construction_KG\output\[banned]\transformed_validset.json", 'w', encoding='utf-8') as f:
#     f.write(json.dumps(trans_validset))
with open(r"G:\python_codes\Principles_of_Computer_Construction_KG\output\[banned]\transformed_testset.json", 'w', encoding='utf-8') as f:
    f.write(json.dumps(trans_testset))

In [29]:
trans_validset[0]

{'sentence': '当一cache行已授权给某处理机核专有，而另外的处理机核企图读此行时，它将送出一个未命中指示给控制器',
 'entities': [{'text': 'cache行', 'start': 2, 'end': 8, 'labels': ['STOR']},
  {'text': '控制器', 'start': 47, 'end': 50, 'labels': ['COMP']},
  {'text': '处理机核', 'start': 13, 'end': 17, 'labels': ['COMP']},
  {'text': '处理机核', 'start': 24, 'end': 28, 'labels': ['COMP']}],
 'relations': [('控制器', 'attribute', '处理机核'), ('cache行', 'contain', '处理机核')]}

In [3]:
# 生成待标注数据集
# 需要合成训练集、验证集、测试集
# trainset: 302
# validset: 50
# testset: 100

with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\trainset\re_pre_annotated_trainset.json', 'r', encoding='utf-8') as f:
    re_trainset = json.loads(f.read())

with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\testset\re_pre_annotated_testset.json', 'r', encoding='utf-8') as f:
    re_validset = json.loads(f.read())

with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\validset\re_pre_annotated_validset.json', 'r', encoding='utf-8') as f:
    re_testset = json.loads(f.read())

In [6]:
len(re_testset)

519

In [7]:
not_annotated_trainset = re_trainset[302:]
not_annotated_validset = re_validset[50:]
not_annotated_testset = re_testset[100:]

In [8]:
not_annotated_trainset[0]

{'data': {'text': '奔腾CPU内的数据cache是一个路组相联结构，就采用这种简捷的LRU替换算法'},
 'predictions': [{'model_version': '',
   'score': 0,
   'result': [{'id': '1204',
     'from_name': 'label',
     'to_name': 'text',
     'type': 'labels',
     'value': {'start': 7,
      'end': 14,
      'score': 0.1,
      'text': '数据cache',
      'labels': ['STOR']}},
    {'id': '1205',
     'from_name': 'label',
     'to_name': 'text',
     'type': 'labels',
     'value': {'start': 32,
      'end': 39,
      'score': 0.1,
      'text': 'LRU替换算法',
      'labels': ['ALG']}},
    {'id': '1206',
     'from_name': 'label',
     'to_name': 'text',
     'type': 'labels',
     'value': {'start': 18,
      'end': 21,
      'score': 0.1,
      'text': '组相联',
      'labels': ['TECH']}},
    {'id': '1207',
     'from_name': 'label',
     'to_name': 'text',
     'type': 'labels',
     'value': {'start': 2,
      'end': 5,
      'score': 0.1,
      'text': 'CPU',
      'labels': ['COMP']}}]}]}

In [16]:
def convert_json_data(data_list):
    converted_data_list = []

    for item in data_list:
        # 提取句子
        sentence = item.get('data', {}).get('text', '')
        
        # 初始化转换后的数据结构
        converted_data = {
            'sentence': sentence,
            'entities': [],
            'relations': []
        }

        # 提取entities
        predictions = item.get('predictions', [])
        if predictions:
            results = predictions[0].get('result', [])
            for result in results:
                entity = {
                    'text': result['value']['text'],
                    'start': result['value']['start'],
                    'end': result['value']['end'],
                    'labels': result['value']['labels']
                }
                converted_data['entities'].append(entity)

        converted_data_list.append(converted_data)

    return converted_data_list

In [19]:
new_not_annotated_trainset = convert_json_data(not_annotated_trainset)
new_not_annotated_validset = convert_json_data(not_annotated_validset)
new_not_annotated_testset = convert_json_data(not_annotated_testset)

In [21]:
with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\[banned]\not_annotated_trainset.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(new_not_annotated_trainset))

with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\[banned]\not_annotated_validset.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(new_not_annotated_validset))

with open(r'G:\python_codes\Principles_of_Computer_Construction_KG\output\[banned]\not_annotated_testset.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(new_not_annotated_testset))